In [210]:
from datetime import datetime
from collections import Counter

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [211]:
if "setup_text_plots" not in globals():
    from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=20, usetex=True)

In [212]:
def format_cat(cat):
    length = len(cat)
    timestamp = np.zeros(length)
    for i in range(length):
        item = cat.iloc[i]
        timestamp[i] = datetime.strptime(item.updateTime, "%Y-%m-%d %H:%M:%S.%f").timestamp() - datetime(2020, 1, 25, 0, 0, 0).timestamp()
    cat = cat.assign(timestamp=timestamp)
    return cat


In [213]:
def cal_daily_province_result(cat):
    provinces = Counter(cat.provinceName).keys()
    day_max = int(np.ceil(np.max(cat.timestamp) / 86400) + 1)
    province_list = list()
    for i in range(day_max):
        timestamp_day = 86400 * i
        item = cat[cat.timestamp < timestamp_day]
        for province in provinces:
            item_sel = item[item.provinceName==province]
            province_dict = {
                    "day": i,
                    "name": province,
                    "date": datetime.fromtimestamp(timestamp_day + datetime(2020, 1, 23, 0, 0, 0).timestamp())
                    }
            if len(item_sel) > 0:
                for key in ['confirmedCount', 'suspectedCount', 'curedCount', 'deadCount']:
                    query = item_sel.sort_values('timestamp', ascending=False).iloc[0]
                    province_dict[key] = query["province_{0}".format(key)]
            else:
                for key in ['confirmedCount', 'suspectedCount', 'curedCount', 'deadCount']:
                    province_dict[key] = 0
            province_list.append(province_dict)
    return pd.DataFrame(province_list)


In [214]:
def cal_daily_nation_result(result_p):
    result_n = dict()
    result_n['day'] = list(Counter(result_p.day).keys())
    dates = list()
    for day in result_n['day']:
        dates.append(result_p[result_p.day==day].iloc[0].date)
    result_n['date'] = dates
    for key in ['confirmedCount', 'suspectedCount', 'curedCount', 'deadCount']:
        count_nation = list()
        count_hubei = list()
        count_nothubei = list()
        for day in Counter(result_p.day).keys():
            item = result_p[result_p.day == day]
            count_nation.append(np.sum(item[key]))
            count_hubei.append(np.sum(item[item.name=='湖北省'][key]))
            count_nothubei.append(np.sum(item[item.name!='湖北省'][key]))
        result_n["{0}_nation".format(key)] = count_nation
        result_n["{0}_hubei".format(key)] = count_hubei
        result_n["{0}_nothubei".format(key)] = count_nothubei
    return pd.DataFrame(result_n)


In [215]:
cat = pd.read_csv('./DXYArea.csv')
cat = format_cat(cat)
result_p = cal_daily_province_result(cat)
result_n = cal_daily_nation_result(result_p)


In [216]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=result_n.date, y=result_n.confirmedCount_hubei,
                    mode='lines',
                    name='hubei'))
fig.add_trace(go.Scatter(x=result_n.date, y=result_n.confirmedCount_nothubei,
                    mode='lines',
                    name='nothubei'))
fig.add_trace(go.Scatter(x=result_n.date, y=result_n.confirmedCount_nation,
                    mode='lines',
                    name='nation'))
fig.update_layout(yaxis_type="log", title="Confirmed Counts", xaxis_title="Date", yaxis_title="N")
fig.show()

In [217]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=result_n.date, y=result_n.deadCount_hubei / result_n.confirmedCount_hubei * 100,
                    mode='lines',
                    name='hubei'))
fig.add_trace(go.Scatter(x=result_n.date, y=result_n.deadCount_nothubei / result_n.confirmedCount_nothubei * 100,
                    mode='lines',
                    name='nothubei'))
fig.add_trace(go.Scatter(x=result_n.date, y=result_n.deadCount_nation / result_n.confirmedCount_nation * 100,
                    mode='lines',
                    name='nation'))
fig.update_layout(title="Death rate I", xaxis_title="Date", yaxis_title="Death Rate (\%)")
fig.show()

In [218]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=result_n.date, y=result_n.deadCount_hubei / result_n.curedCount_hubei * 100,
                    mode='lines',
                    name='hubei'))
fig.add_trace(go.Scatter(x=result_n.date, y=result_n.deadCount_nothubei / result_n.curedCount_nothubei * 100,
                    mode='lines',
                    name='nothubei'))
fig.add_trace(go.Scatter(x=result_n.date, y=result_n.deadCount_nation / result_n.curedCount_nation * 100,
                    mode='lines',
                    name='nation'))
fig.update_layout(title="Death rate II", xaxis_title="Date", yaxis_title="Death Rate (\%)")
fig.show()

In [219]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=result_n.date.loc[1:], y=result_n.confirmedCount_hubei.values[1:] - result_n.confirmedCount_hubei.values[:-1],
                    mode='lines',
                    name='hubei'))
fig.add_trace(go.Scatter(x=result_n.date.loc[1:], y=result_n.confirmedCount_nothubei.values[1:] - result_n.confirmedCount_nothubei[:-1],
                    mode='lines',
                    name='nothubei'))
fig.add_trace(go.Scatter(x=result_n.date.loc[1:], y=result_n.confirmedCount_nation.values[1:] - result_n.confirmedCount_nation.values[:-1],
                    mode='lines',
                    name='nation'))
fig.update_layout(title="Daily Confirmed Counts Increase", xaxis_title="Date", yaxis_title="N")
fig.show()

In [230]:
fig = go.Figure()
for province in provinces:
    result = result_p[result_p['name']==province]
    if np.max(result.confirmedCount) > 200:
        ax.plot(result.date, result.confirmedCount / np.max(result.confirmedCount))
        fig.add_trace(go.Scatter(x=result.date, y=result.confirmedCount / np.max(result.confirmedCount),
                    mode='lines',
                    name=province))
fig.update_layout(title="Cumulative Confirmed Counts", xaxis_title="Date", yaxis_title="N")
fig.update_yaxes(range=[0.4, 1])
fig.update_xaxes(range=[result.date.iloc[-12], result.date.iloc[-1]])
fig.show()